In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / 'src'))

from phise import Context
from phise.classes.companion import Companion
from src.analysis.data_representations import data_representations
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

# Configuration
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
print("✓ Imports effectués")

## Configuration du Contexte

Nous configurons un contexte VLTI avec :
- 4 télescopes en configuration nuller
- Une compagne faible (contraste ~1%) avec séparation angulaire
- Erreurs de piston réalistes (Γ = 5 nm)

In [ ]:
# Configuration du contexte
ctx = Context.get_VLTI()

# Ajouter une compagne pour l'analyse
if not ctx.target.companions:
    companion = Companion(Δα=150*u.mas, Δδ=100*u.mas, c=0.01)  # Contraste 1%
    ctx.target.companions = [companion]

# Configurer les paramètres d'observation
ctx.interferometer.chip.σ = np.zeros(14) * u.nm  # Pas d'aberrations
ctx.Γ = 5 * u.nm  # Erreur de piston petite mais réaliste

print("Contexte configuré :")
print(f"  ✓ Télescopes : {len(ctx.interferometer.telescopes)}")
print(f"  ✓ Longueur d'onde : {ctx.interferometer.λ}")
print(f"  ✓ Contraste compagne : {ctx.target.companions[0].c if ctx.target.companions else 'N/A'}")
print(f"  ✓ Erreur de piston (Γ) : {ctx.Γ}")
print(f"  ✓ Bande spectrale : {ctx.interferometer.Δλ}")

## Distribution Instantanée

La distribution instantanée représente l'histogramme des sorties du nuller pour un nombre d'observations donné.
Elle permet de comparer :
- Le modèle **analytique** (formules fermées basées sur les matrices de transmission)
- Le modèle **numérique** (simulation complète de la propagation optique)
- Les différents scénarios : étoile seule, planète seule, et complet

### Hypothèse
Les erreurs de piston sont distribuées selon une gaussienne centrée : $\\Delta OPD \\sim \\mathcal{N}(0, \\Gamma^2)$

In [ ]:
# Calcul de la distribution instantanée
# Cela génère plusieurs figures comparant le modèle analytique et numérique
print("Calcul de la distribution instantanée...")
data, data_so = data_representations.instant_distribution(
    ctx=ctx,
    n=5000,  # Nombre d'observations
    stat=np.median,  # Statistique à utiliser
    figsize=(14, 10),
    compare=True,  # Comparer analytique et numérique
    sync_plots=True,  # Synchroniser les axes
    show=True,
    density=False,
    log=False
)
print("✓ Distribution instantanée calculée")

## Analyse de la Distribution

### Observations Clés

1. **Symétrie des distributions** : Pour l'étoile seule, les noyaux sont centrés à zéro (destructive interférence parfaite)
2. **Bruit analytique vs numérique** : Les deux modèles devraient montrer une bonne concordance
3. **Effet de la planète** : La compagne provoque une modulation des noyaux dépendant de :
   - Sa séparation angulaire
   - Son contraste
   - Sa longueur d'onde

### Signal de Détection

Pour un système avec étoile + compagne, on observe :
$$K(\\alpha, \\delta) = K_{star} + K_{planet} \\approx K_{planet}$$

car $K_{star} \\approx 0$ en optimal nulling.

## Évolution Temporelle

L'évolution temporelle analyse comment les sorties du nuller varient avec le changement d'angle horaire.
Cela simule l'observation d'une source pendant sa trajectoire dans le ciel.

### Phénomènes Observés
- **Variation du null** : Due à la géométrie du réseau qui change avec l'heure
- **Turbulence atmosphérique** : Cause des fluctuations aléatoires
- **Dispersion spectrale** : Différentes longueurs d'onde produisent des noyaux différents

In [ ]:
# Évolution temporelle
print("Calcul de l'évolution temporelle...")
data_time, ref_data = data_representations.time_evolution(
    ctx=ctx,
    n=100,  # Nombre d'observations par angle horaire
    map=np.median,  # Statistique pour chaque angle
    show=True
)
print(f"✓ Évolution temporelle calculée")
print(f"  Forme des données : {data_time.shape}")
print(f"  Nombre d'angles horaires : {data_time.shape[0]}")
print(f"  Nombre de noyaux : {data_time.shape[1]}")

## Comparaison des Statistiques

Affichage détaillé de la comparaison entre le modèle analytique et les simulations numériques.

In [ ]:
# Afficher des statistiques détaillées
print("\\n" + "="*60)
print("STATISTIQUES DES DISTRIBUTIONS")
print("="*60)

# Analyse de la distribution temporelle
print(f"\\nDistribution Temporelle:")
print(f"  Kernel 1: μ={np.mean(data_time[:, 0]):.2e}, σ={np.std(data_time[:, 0]):.2e}")
print(f"  Kernel 2: μ={np.mean(data_time[:, 1]):.2e}, σ={np.std(data_time[:, 1]):.2e}")
print(f"  Kernel 3: μ={np.mean(data_time[:, 2]):.2e}, σ={np.std(data_time[:, 2]):.2e}")

# Référence
print(f"\\nRéférence (sans bruit):")
print(f"  Kernel 1: {ref_data[0, 0]:.2e}")
print(f"  Kernel 2: {ref_data[0, 1]:.2e}")
print(f"  Kernel 3: {ref_data[0, 2]:.2e}")
print("\\n" + "="*60)